In [1]:
from functions import *
import os
import unittest
forward = True
iri = False
qualified_name = {
    "xsd:QName",
    "prov:QUALIFIED_NAME"
}
path_to_json = '/Users/cuiyeshuai/Documents/UG modules/submission/openprov/'
json_file = '/Users/cuiyeshuai/Documents/UG modules/submission/datasets/CM-Buildings/Building729.0.json'

In [ ]:
f = open(json_file, 'r')
data = f.read()
f.close()
prov = ProvDocument()
#prov = document_to_encoding(ProvDocument.deserialize(json_file), iri = iri, forward = forward)
prov = json_to_encoding(data, iri = iri, forward = forward, qualified_name = qualified_name)
print(prov)

In [ ]:
for file_name in [file for file in os.listdir(path_to_json) if file.endswith('.json')]:
    with open(path_to_json + file_name) as json_file:
      print(file_name)  
      data = json_file.read()
      prov = document_to_encoding(ProvDocument.deserialize(content=data), iri = iri, forward = forward)
      prov1 = json_to_encoding(data, iri = iri, forward = forward, qualified_name = qualified_name)
      print(prov)

In [39]:
class TestFunctions(unittest.TestCase):
    def test_node_and_edge_number(self):
        f = open(json_file, 'r')
        data = f.read()
        f.close()
        prov_encoding = json_to_encoding(data, iri = iri, forward = forward, qualified_name = qualified_name)
        node_number = len(list(prov_encoding[0].keys()))
        edge_number = 0
        for start in prov_encoding[1]:
            for end in prov_encoding[1][start]:
                edge_number += 1
        file = json.decoder.JSONDecoder().decode(data)
        node_number_file = 0
        edge_number_file = 0
        for rec_type in PROV_NODE:
            if file.get(rec_type) is not None:
                for x in file.get(rec_type):
                    node_number_file += 1
        for rec_type in PROV_EDGE:
            if file.get(rec_type) is not None:
                for x in file.get(rec_type):
                    edge_number_file += 1
        self.assertEqual(node_number, node_number_file)
        self.assertEqual(edge_number, edge_number_file)
    
    def test_node_label(self):
        f = open(json_file, 'r')
        data = f.read()
        f.close()
        prov_encoding = json_to_encoding(data, iri = iri, forward = forward, qualified_name = qualified_name)
        self.assertEqual(prov_encoding[0]["BuildingVerification463"][0], "activity")
        self.assertEqual(prov_encoding[0]["BuildingVerification463"][1], frozenset(["collabmap:BuildingVerification"]))
        self.assertEqual(prov_encoding[1]["BuildingVerification463"][0], "used")
    
    def test_equal_encoding(self):
        f = open(json_file, 'r')
        data = f.read()
        f.close()
        prov = ProvDocument()
        prov = document_to_encoding(ProvDocument.deserialize(json_file), iri = iri, forward = forward)
        prov1 = json_to_encoding(data, iri = iri, forward = forward, qualified_name = qualified_name)
        self.assertEqual(prov, prov1)
    # def test_equal_encoding_all(self):
    #     for file_name in [file for file in os.listdir(path_to_json) if file.endswith('0.json')]:
    #         with open(path_to_json + file_name) as json_file:              
    #             data = json_file.read()
    #             prov = document_to_encoding(ProvDocument.deserialize(content=data), iri = iri, forward = forward)
    #             prov1 = json_to_encoding(data, iri = iri, forward = forward, qualified_name = qualified_name)
    #             with self.subTest(prov = prov, prov1 = prov1):
    #                 self.assertEqual(prov, prov1)
    

In [40]:
unittest.main(argv=[''], exit=False,verbosity=2)

test_equal_encoding (__main__.TestFunctions) ... ok
test_node_and_edge_number (__main__.TestFunctions) ... ok
test_node_label (__main__.TestFunctions) ... FAIL

FAIL: test_node_label (__main__.TestFunctions)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/kk/6809ldm97b5c5tnp1p0hcql00000gn/T/ipykernel_89523/3120841511.py", line 33, in test_node_label
    self.assertEqual(prov_encoding[1]["BuildingVerification463"][0], "used")
AssertionError: ('Building729.0', ('used', frozenset())) != 'used'

----------------------------------------------------------------------
Ran 3 tests in 0.008s

FAILED (failures=1)
